In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.1 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0605.csv		       Github
 a0608.csv		       heatmap529_UK.ipynb
 a0609.csv		       heatmap530.ipynb
 ccc_compiled.csv	       Heat_Map_Online_language.ipynb
 csv			       __MACOSX
 csv.zip		       mapping_UK
 dataframe_csv		       mapping_US
 dataframe_csv_EU	       non_violant1.csv
 dataframe_csv_UK	       non_violant2.csv
 dataframe_result_529_UK.csv   UK
 dataframe_result_530_UK.csv   violant.csv
 dataframe_result_531_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_csv


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct=pd.read_csv('a0529-a0611_UK/a0611.csv',encoding='ISO-8859-1')

In [8]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0605.csv		       Github
 a0608.csv		       heatmap529_UK.ipynb
 a0609.csv		       heatmap530.ipynb
 ccc_compiled.csv	       Heat_Map_Online_language.ipynb
 csv			       __MACOSX
 csv.zip		       mapping_UK
 dataframe_csv		       mapping_US
 dataframe_csv_EU	       non_violant1.csv
 dataframe_csv_UK	       non_violant2.csv
 dataframe_result_529_UK.csv   UK
 dataframe_result_530_UK.csv   violant.csv
 dataframe_result_531_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_csv


In [9]:
###
#ct=pd.read_csv('a0601.csv')

In [10]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,119550118426654_1144228372625485,Facebook,2020-06-11 23:59:26,2021-09-01 11:27:10,video,Peaceful Protests Spark in Wyandanch and Brent...,shadesoflongisland.com,Please take 8:46 to watch this film. DONâT L...,Take the time it took to kill George Floyd and...,1.0,https://shadesoflongisland.com/2020/06/01/peac...,https://www.facebook.com/groups/11955011842665...,0,-0.277778,1.0,6,6,1,0,0,0,0,0,0,2,5,2,2,2,0,0,6,1,0,2,12884237,Race Unity Group of Carbondale,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,NaN,NaN,False,NaN,en,0,12884237|1144228372625485,{},{},"{'Wyandanch': 1, 'Brentwood': 1, 'Long Island'..."
1,1,352050521888601_911053385988309,Facebook,2020-06-11 23:59:24,2021-07-26 15:24:48,link,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latsonâs case shows that while some losses o...,Neli Latsonâs life changed in the same swift...,1.0,https://www.washingtonpost.com/local/remember-...,https://www.facebook.com/groups/35205052188860...,0,-0.916667,1.0,0,0,0,0,0,0,1,0,0,0,2,1,3,1,1,2,1,1,0,1,15825951,The Truth!!!,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,NaN,NaN,False,NaN,en,0,15825951|911053385988309,{'Virginia': 1},{},{}
2,2,179948425230_10163588345370231,Facebook,2020-06-11 23:59:20,2021-07-09 21:53:15,native_video,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latsonâs case shows that while some losses o...,Talking about next steps for justice live on @...,1.0,https://www.facebook.com/realsharpton/videos/2...,https://www.facebook.com/179948425230/posts/10...,597764,2.771739,2.0,291,90,52,71,2,1,1,2,0,18,106,29,20,18,2,4,3,2,0,5,48824,Reverend Al Sharpton,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,FOLLOW ME ON TWITTER! www.twitter.com/TheRevAl...,True,2009-05-26 00:56:53,en,0,48824|10163588345370231,{},{},{}
3,3,245354138137_10157682143678138,Facebook,2020-06-11 23:59:18,2021-03-02 18:50:37,link,UUC Gathers Online,mailchi.mp,I write today with a heavy heart over the deat...,UUC Gathers Online - https://mailchi.mp/d91def...,2.0,https://mailchi.mp/d91def54f14b/8pcnwx8zix-421...,https://www.facebook.com/245354138137/posts/10...,374,-0.857143,1.0,1,0,0,0,0,0,0,0,0,0,2,2,2,1,0,0,0,0,0,0,223616,Unitarian Universalist Congregation in Stamfor...,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,We exist to build a community that deepens ind...,False,2010-01-14 16:49:39,en,0,223616|10157682143678138,{},{'Minneapolis': 1},{}
4,4,250147468771235_982208315565143,Facebook,2020-06-11 23:59:17,2021-09-07 01:05:13,link,Biden says Floyd death having bigger global im...,apple.news,Former Vice President Joe Biden on Thursday sa...,Right when you think Joe Biden can't get anymo...,1.0,https://apple.news/AduJeEjRNQPOBcyKxx_62jg,https://www.facebook.com/groups/25014746877123...,876,-0.764706,1.0,0,1,3,0,0,0,0,0,0,0,2,2,4,1,2,2,2,2,0,1,10779034,Bronx Minds on Politics,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,We exist to build a community that deepens ind...,False,2010-01-14 16:49:39,en,0,10779034|982208315565143,{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [11]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [12]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [13]:
###
ct_ner = ct

In [14]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{},"{'Wyandanch': 1, 'Brentwood': 1, 'Long Island'..."
1,{'Virginia': 1},{},{}
2,{},{},{}
3,{},{'Minneapolis': 1},{}
4,{},{},{}


In [15]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [16]:
ct_ner['NER-msg']

0                 []
1         [Virginia]
2                 []
3                 []
4                 []
            ...     
15153             []
15154      [Houston]
15155             []
15156    [Minnesota]
15157             []
Name: NER-msg, Length: 15158, dtype: object

In [17]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [18]:
ct_ner['NER']

0          [Brentwood, Wyandanch, Long Island]
1                                   [Virginia]
2                                           []
3                                [Minneapolis]
4                                           []
                         ...                  
15153                                       []
15154                                [Houston]
15155               [Minneapolis, San Antonio]
15156    [Minnesota, Minneapolis, San Antonio]
15157                                       []
Name: NER, Length: 15158, dtype: object

In [19]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [20]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [21]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,119550118426654_1144228372625485,Facebook,2020-06-11 23:59:26,2021-09-01 11:27:10,video,Peaceful Protests Spark in Wyandanch and Brent...,shadesoflongisland.com,Please take 8:46 to watch this film. DONâT L...,Take the time it took to kill George Floyd and...,1.0,https://shadesoflongisland.com/2020/06/01/peac...,https://www.facebook.com/groups/11955011842665...,0,-0.277778,1.0,6,6,1,0,0,0,0,0,0,2,5,2,2,2,0,0,6,1,0,2,12884237,Race Unity Group of Carbondale,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,NaN,NaN,False,NaN,en,0,12884237|1144228372625485,[],[],"[Brentwood, Wyandanch, Long Island]","[brentwood, wyandanch, long island]"
1,1,352050521888601_911053385988309,Facebook,2020-06-11 23:59:24,2021-07-26 15:24:48,link,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latsonâs case shows that while some losses o...,Neli Latsonâs life changed in the same swift...,1.0,https://www.washingtonpost.com/local/remember-...,https://www.facebook.com/groups/35205052188860...,0,-0.916667,1.0,0,0,0,0,0,0,1,0,0,0,2,1,3,1,1,2,1,1,0,1,15825951,The Truth!!!,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,NaN,NaN,False,NaN,en,0,15825951|911053385988309,[Virginia],[],[],[virginia]
2,2,179948425230_10163588345370231,Facebook,2020-06-11 23:59:20,2021-07-09 21:53:15,native_video,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latsonâs case shows that while some losses o...,Talking about next steps for justice live on @...,1.0,https://www.facebook.com/realsharpton/videos/2...,https://www.facebook.com/179948425230/posts/10...,597764,2.771739,2.0,291,90,52,71,2,1,1,2,0,18,106,29,20,18,2,4,3,2,0,5,48824,Reverend Al Sharpton,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,FOLLOW ME ON TWITTER! www.twitter.com/TheRevAl...,True,2009-05-26 00:56:53,en,0,48824|10163588345370231,[],[],[],[]
3,3,245354138137_10157682143678138,Facebook,2020-06-11 23:59:18,2021-03-02 18:50:37,link,UUC Gathers Online,mailchi.mp,I write today with a heavy heart over the deat...,UUC Gathers Online - https://mailchi.mp/d91def...,2.0,https://mailchi.mp/d91def54f14b/8pcnwx8zix-421...,https://www.facebook.com/245354138137/posts/10...,374,-0.857143,1.0,1,0,0,0,0,0,0,0,0,0,2,2,2,1,0,0,0,0,0,0,223616,Unitarian Universalist Congregation in Stamfor...,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,We exist to build a community that deepens ind...,False,2010-01-14 16:49:39,en,0,223616|10157682143678138,[],[Minneapolis],[],[minneapolis]
4,4,250147468771235_982208315565143,Facebook,2020-06-11 23:59:17,2021-09-07 01:05:13,link,Biden says Floyd death having bigger global im...,apple.news,Former Vice President Joe Biden on Thursday sa...,Right when you think Joe Biden can't get anymo...,1.0,https://apple.news/AduJeEjRNQPOBcyKxx_62jg,https://www.facebook.com/groups/25014746877123...,876,-0.764706,1.0,0,1,3,0,0,0,0,0,0,0,2,2,4,1,2,2,2,2,0,1,10779034,Bronx Minds on Politics,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,We exist to build a community that deepens ind...,False,2010-01-14 16:49:39,en,0,10779034|982208315565143,[],[],[],[]
...,...,...,...,...,...,...,...,...,.

In [22]:
ct_ner['NER-1']=ct_ner['NER']

In [23]:
ct_ner['NER-1'][1]

['virginia']

In [24]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 5.1 MB/s 


In [25]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [26]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,119550118426654_1144228372625485,Facebook,2020-06-11 23:59:26,2021-09-01 11:27:10,video,Peaceful Protests Spark in Wyandanch and Brent...,shadesoflongisland.com,Please take 8:46 to watch this film. DONâT L...,Take the time it took to kill George Floyd and...,1.0,https://shadesoflongisland.com/2020/06/01/peac...,https://www.facebook.com/groups/11955011842665...,0,-0.277778,1.0,6,6,1,0,0,0,0,0,0,2,5,2,2,2,0,0,6,1,0,2,12884237,Race Unity Group of Carbondale,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,NaN,NaN,False,NaN,en,0,12884237|1144228372625485,[],[],"[Brentwood, Wyandanch, Long Island]","[brentwood, wyandanch, long island]","[brentwood, wyandanch, long island]"
1,1,352050521888601_911053385988309,Facebook,2020-06-11 23:59:24,2021-07-26 15:24:48,link,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latsonâs case shows that while some losses o...,Neli Latsonâs life changed in the same swift...,1.0,https://www.washingtonpost.com/local/remember-...,https://www.facebook.com/groups/35205052188860...,0,-0.916667,1.0,0,0,0,0,0,0,1,0,0,0,2,1,3,1,1,2,1,1,0,1,15825951,The Truth!!!,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,NaN,NaN,False,NaN,en,0,15825951|911053385988309,[Virginia],[],[],[virginia],[virginia]
2,2,179948425230_10163588345370231,Facebook,2020-06-11 23:59:20,2021-07-09 21:53:15,native_video,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latsonâs case shows that while some losses o...,Talking about next steps for justice live on @...,1.0,https://www.facebook.com/realsharpton/videos/2...,https://www.facebook.com/179948425230/posts/10...,597764,2.771739,2.0,291,90,52,71,2,1,1,2,0,18,106,29,20,18,2,4,3,2,0,5,48824,Reverend Al Sharpton,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,FOLLOW ME ON TWITTER! www.twitter.com/TheRevAl...,True,2009-05-26 00:56:53,en,0,48824|10163588345370231,[],[],[],[],[]
3,3,245354138137_10157682143678138,Facebook,2020-06-11 23:59:18,2021-03-02 18:50:37,link,UUC Gathers Online,mailchi.mp,I write today with a heavy heart over the deat...,UUC Gathers Online - https://mailchi.mp/d91def...,2.0,https://mailchi.mp/d91def54f14b/8pcnwx8zix-421...,https://www.facebook.com/245354138137/posts/10...,374,-0.857143,1.0,1,0,0,0,0,0,0,0,0,0,2,2,2,1,0,0,0,0,0,0,223616,Unitarian Universalist Congregation in Stamfor...,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,We exist to build a community that deepens ind...,False,2010-01-14 16:49:39,en,0,223616|10157682143678138,[],[Minneapolis],[],[minneapolis],[minneapolis]
4,4,250147468771235_982208315565143,Facebook,2020-06-11 23:59:17,2021-09-07 01:05:13,link,Biden says Floyd death having bigger global im...,apple.news,Former Vice President Joe Biden on Thursday sa...,Right when you think Joe Biden can't get anymo...,1.0,https://apple.news/AduJeEjRNQPOBcyKxx_62jg,https://www.facebook.com/groups/25014746877123...,876,-0.764706,1.0,0,1,3,0,0,0,0,0,0,0,2,2,4,1,2,2,2,2,0,1,10779034,Bronx Minds on Politics,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,We exist to build a community that deepens ind...,False,2010-01-14 16:49:39,en,0,10

In [27]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [28]:
!ls

dataframe0529_post.csv	dataframe0607_post.csv	    dataframe_mapping_0603.csv
dataframe0530_post.csv	dataframe0608_post.csv	    dataframe_mapping_0604.csv
dataframe0531_post.csv	dataframe0609_post.csv	    dataframe_mapping_0605.csv
dataframe0601_post.csv	dataframe0610_post.csv	    dataframe_mapping_0606.csv
dataframe0602_post.csv	dataframe_mapping_0529.csv  dataframe_mapping_0607.csv
dataframe0603_post.csv	dataframe_mapping_0530.csv  dataframe_mapping_0608.csv
dataframe0604_post.csv	dataframe_mapping_0531.csv  dataframe_mapping_0609.csv
dataframe0605_post.csv	dataframe_mapping_0601.csv  dataframe_mapping_0610.csv
dataframe0606_post.csv	dataframe_mapping_0602.csv  dataframe_mapping_0611.csv


In [29]:
ct_mapping=pd.read_csv('dataframe_mapping_0611.csv',encoding='ISO-8859-1')

In [30]:
len(ct_mapping['City'])

2359

In [31]:
len(ct_ner['NER-1'])

15158

In [32]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [33]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,119550118426654_1144228372625485,Facebook,2020-06-11 23:59:26,2021-09-01 11:27:10,video,Peaceful Protests Spark in Wyandanch and Brent...,shadesoflongisland.com,Please take 8:46 to watch this film. DONâT L...,Take the time it took to kill George Floyd and...,1.0,https://shadesoflongisland.com/2020/06/01/peac...,https://www.facebook.com/groups/11955011842665...,0,-0.277778,1.0,6,6,1,0,0,0,0,0,0,2,5,2,2,2,0,0,6,1,0,2,12884237,Race Unity Group of Carbondale,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,NaN,NaN,False,NaN,en,0,12884237|1144228372625485,[],[],"[Brentwood, Wyandanch, Long Island]","[brentwood, wyandanch, long island]","[Brentwood, Contra Costa County, California, 9..."
1,1,352050521888601_911053385988309,Facebook,2020-06-11 23:59:24,2021-07-26 15:24:48,link,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latsonâs case shows that while some losses o...,Neli Latsonâs life changed in the same swift...,1.0,https://www.washingtonpost.com/local/remember-...,https://www.facebook.com/groups/35205052188860...,0,-0.916667,1.0,0,0,0,0,0,0,1,0,0,0,2,1,3,1,1,2,1,1,0,1,15825951,The Truth!!!,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,NaN,NaN,False,NaN,en,0,15825951|911053385988309,[Virginia],[],[],[virginia],"[Virginia, United States]"
2,2,179948425230_10163588345370231,Facebook,2020-06-11 23:59:20,2021-07-09 21:53:15,native_video,"Perspective | Remember Neli Latson, the black ...",washingtonpost.com,Latsonâs case shows that while some losses o...,Talking about next steps for justice live on @...,1.0,https://www.facebook.com/realsharpton/videos/2...,https://www.facebook.com/179948425230/posts/10...,597764,2.771739,2.0,291,90,52,71,2,1,1,2,0,18,106,29,20,18,2,4,3,2,0,5,48824,Reverend Al Sharpton,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,FOLLOW ME ON TWITTER! www.twitter.com/TheRevAl...,True,2009-05-26 00:56:53,en,0,48824|10163588345370231,[],[],[],[],[]
3,3,245354138137_10157682143678138,Facebook,2020-06-11 23:59:18,2021-03-02 18:50:37,link,UUC Gathers Online,mailchi.mp,I write today with a heavy heart over the deat...,UUC Gathers Online - https://mailchi.mp/d91def...,2.0,https://mailchi.mp/d91def54f14b/8pcnwx8zix-421...,https://www.facebook.com/245354138137/posts/10...,374,-0.857143,1.0,1,0,0,0,0,0,0,0,0,0,2,2,2,1,0,0,0,0,0,0,223616,Unitarian Universalist Congregation in Stamfor...,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,We exist to build a community that deepens ind...,False,2010-01-14 16:49:39,en,0,223616|10157682143678138,[],[Minneapolis],[],[minneapolis],"[Minneapolis, Hennepin County, Minnesota, 5541..."
4,4,250147468771235_982208315565143,Facebook,2020-06-11 23:59:17,2021-09-07 01:05:13,link,Biden says Floyd death having bigger global im...,apple.news,Former Vice President Joe Biden on Thursday sa...,Right when you think Joe Biden can't get anymo...,1.0,https://apple.news/AduJeEjRNQPOBcyKxx_62jg,https://www.facebook.com/groups/25014746877123...,876,-0.764706,1.0,0,1,3,0,0,0,0,0,0,0,2,2,4,1,2,2,2,2,0,1,10779034,Bronx Minds on Politics,realsharpton,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,We exist to bu

In [34]:
ct_ner['NER-1'][12]

['Fort Worth, Tarrant County, Texas, 76102, United States',
 'Texas, United States',
 nan]

In [35]:
ct_ner.to_csv('dataframe0611_post.csv')